In [ ]:
import sqlite3

db_path = r"Output/database/unified_database.db"
conn = sqlite3.connect(db_path)

trajs = pd.read_sql("SELECT SegmentId, CrossingSpeedMph FROM trajs;", conn)
seg2link = pd.read_sql("SELECT SegmentId, link_id FROM SegmentId_to_link;", conn)

# Unir
trajs_link = trajs.merge(seg2link, on='SegmentId', how='inner')

# Promediar por link_id
trajs_link = trajs_link.groupby('link_id', as_index=False)['CrossingSpeedMph'].mean()
trajs_link.rename(columns={'CrossingSpeedMph': 'TrajectorySpeed'}, inplace=True)

trajs_link.head()



df = wp_link.merge(trajs_link, on='link_id', how='inner')

y_true = df['TrajectorySpeed']       # velocidad de trajs
y_pred = df['WaypointSpeed']         # velocidad waypoint map-matched



from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import numpy as np

rmse = np.sqrt(mean_squared_error(y_true, y_pred))
mae  = mean_absolute_error(y_true, y_pred)
mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
r2   = r2_score(y_true, y_pred)

rmse, mae, mape, r2



import matplotlib.pyplot as plt

plt.figure(figsize=(8,6))
plt.scatter(y_pred, y_true, alpha=0.6, color='salmon', edgecolor='k')

# 1:1 line
min_val = min(y_true.min(), y_pred.min())
max_val = max(y_true.max(), y_pred.max())
plt.plot([min_val, max_val], [min_val, max_val], 'r--', label="1:1 Line")

plt.xlabel("Waypoint Map-Matched Speed (mph)")
plt.ylabel("Trajectory Speed (mph)")
plt.title("Cross-Source Speed Validation (Waypoint vs Trajs)")

plt.text(max_val - 20, min_val + 5,
         f"RMSE: {rmse:.2f} mph\nMAE: {mae:.2f} mph\nMAPE: {mape:.2f}%\nR²: {r2:.3f}",
         bbox=dict(facecolor='white', alpha=0.7))

plt.grid(True, linestyle='--', alpha=0.4)
plt.show()
